In [ ]:
import re
import torch
import random
import pandas as pd
from langchain import PromptTemplate, FewShotPromptTemplate
from datasets import load_dataset
from pathlib import Path

import sys
if ".." not in sys.path:
    sys.path.append("..")

from lit_llama import LLaMA, Tokenizer

In [ ]:
tokenizer_path = Path("../checkpoints/lit-llama/tokenizer.model")
tokenizer = Tokenizer(tokenizer_path)

In [ ]:
tokenizer.encode("\n")

In [ ]:
tokenizer.decode(torch.tensor([13]))

In [ ]:
tgt_lang = "en"
src = "München 1856: Vier Karten, die Ihren Blick auf die Stadt verändern"
text = "de: Gestern 07:01 TubeWolf kleine tittchen, magersüchtig, doggy style, brünette, schönheit, erotik, reiten\nen:Yesterday 07:01 TubeWolf small tits, skinny, dogging, brunette, beauty, erotic, riding\nde: Uhr Visby fü Ihre Webseite\nen:Get Visby Clock for your website\nde: München 1856: Vier Karten, die Ihren Blick auf die Stadt verändern\nen:Munich 1856: Four cards that change the way you see the city\nde:shds"
# match everything between "de:" and "\n$"


re.search(f"{src}\n{tgt_lang}:(.*)\n",text).group(1)

In [ ]:

dataset = load_dataset("wmt19","de-en")

In [ ]:
import random
sid = random.sample(range(len(dataset["train"])), k=5)

In [ ]:
dataset["train"][0]

In [ ]:
pair = "de-en"
src_lang, tgt_lang = pair.split("-")

In [ ]:


df = pd.read_csv("./data/wmt19_en-de_translation.csv")

In [ ]:
from sacrebleu import sentence_bleu, sentence_chrf


In [ ]:
df['bleu'] = [sentence_bleu(hyp, [ref]).score for hyp, ref
                              in zip(df['mt'],
                                     df['ref'])]
df['chrf'] = [sentence_chrf(hyp, [ref]).score for hyp, ref
                              in zip(df['mt'],
                                     df['ref'])]

In [ ]:
data = [
    {
        "src":src,
        "ref":ref,
        "mt":mt,
    }
    for src, ref, mt in zip(df['src'],df['ref'],df['mt'])
]

In [ ]:
from comet import download_model, load_from_checkpoint

model_path = download_model("Unbabel/wmt22-comet-da")
model = load_from_checkpoint(model_path)
model_output = model.predict(data, batch_size=8, gpus=1)
df['wmt22-comet-da'] = model_output.scores

In [ ]:
df

In [ ]:
# average the columns bleu, chrf and wmt22-comet-da
results = {
    "translated":".mt/data/wmt19_en-de_translated.csv",
    "bleu":df['bleu'].mean(),
    "chrf":df['chrf'].mean(),
    "wmt22-comet-da":df['wmt22-comet-da'].mean(),
}
# save dict to .json file
import json
with open("./mt/wmt19_en-de_results.json", "w") as f:
    json.dump(results, f, indent=4)

In [ ]:
df_results